In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from matplotlib import pyplot as plt

Código do Projeto

In [ ]:
def accuracy(predictions, labels):
    return int(sum(labels == predictions) / len(labels) * 100)

def normalize(data, feature_range=(0, 1)):

    data = np.array(data)
    scaler = MinMaxScaler(feature_range=feature_range)
    normalized_data = scaler.fit_transform(data)
    return normalized_data, scaler


def denormalize(normalized_data, scaler):

    normalized_data = np.array(normalized_data)
    denormalized_data = scaler.inverse_transform(normalized_data)

    return denormalized_data

def evaluate_regression(targets, predictions):

    targets = np.array(targets)
    predictions = np.array(predictions)

    # Compute metrics
    mse = mean_squared_error(targets, predictions)  # Mean Squared Error
    rmse = np.sqrt(mse)                    # Root Mean Squared Error
    r2 = r2_score(targets, predictions)  # R^2 Score

    # Return metrics in a dictionary
    return {"RMSE": rmse, "R^2": r2}
    
def splitData(features, labels, validationPercentage=0.2):
    featuresTrain, featuresVal, targetTrain, targetVal = train_test_split(
        features,
        labels,
        test_size=validationPercentage,
    )
    return featuresTrain, featuresVal, targetTrain, targetVal



def evaluate_classification(labels, predictions):
    labels = np.array(labels)
    predictions = np.array(predictions)

    metrics = {
        "Accuracy": accuracy_score(labels, predictions),
        "Precision": precision_score(labels, predictions),
        "Recall": recall_score(labels, predictions),
        "F1 Score": f1_score(labels, predictions),

    }
    return metrics


def evaluate_multiclass_with_one_hot(labelOneHot, predictionOneHot):

    # Converte one-hot labels para indices de classes
    label = np.argmax(labelOneHot, axis=1)
    prediction = np.argmax(predictionOneHot, axis=1)

    # Computa as métricas
    metrics = {
        'accuracy': accuracy_score(label, prediction),
        'precision': precision_score(label, prediction, average='weighted', zero_division=0),
    }

    print("Evaluation Metrics:")

    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")

    return metrics


#Activation functions

def sigmoid(x):
    #Função de ativação usada para normalizar valores em um intervalo de 0 a 1
    return 1/(1 + np.exp(-x))

def sigmoidDerivative(x):
    #Calcula a derivada da função sigmoide, útil para o backpropagation
    sigmoidResult = sigmoid(x)
    return sigmoidResult * (1 - sigmoidResult)


def identity(x):
    #Função de ativação que não modifica a saída do neurônio, útil em tarefas de regressão 
    return x

def identityDerivative(x):
    #Derivada da função identidade
    return np.ones_like(x)

def ReLU(x):
    #Função de ativação usada para introduzir não-linearidade. Muito utilizada para neurônios de camadas ocultas.
    return np.maximum(0, x)

def ReluDerivative(x):
    #Calcula a derivada de ReLU
    return np.where(x > 0, 1, 0)

def softmax(x):
    # usar para classificação multi-classes

    x = x - np.max(x, axis=-1, keepdims=True)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def softmaxDerivative(x):
    # Em razão da fórmula de cálculo para cost da categoricalEntropy, não é necessário calcular a derivada da softmax
    return 1


activationFunctions = {
    "SIGMOID": (sigmoid, sigmoidDerivative),
    "RELU": (ReLU, ReluDerivative),
    "SOFTMAX": (softmax, softmaxDerivative),
    "IDENTITY": (identity, identityDerivative)
}

# Cost functions

def meanSquaredError(predictions, labels):
    # Função de custo erro quadrático. Usar para regressão
    return np.mean((predictions - labels) ** 2)


def mseDerivative(prediction, label):
    # Derivada da mse para uma observação
    return 2 * (prediction - label)


def binaryCrossEntropy(predictions, labels):
    # Calcula o erro de entropia cruzada. Usar para classificação binária

    epsilon=1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon) #limita valores de prediction para evitar logaritmos de 0 ou 1, que resultariam em erros.

    return -np.mean(labels * np.log(predictions) + (1 - labels) * np.log(1 - predictions))


def binaryEntropyDerivative(prediction, label):
    #Calcula a derivada da entropia cruzada binária
    epsilon=1e-15
    prediction = np.clip(prediction, epsilon, 1 - epsilon) #limita valores de prediction para evitar logaritmos de 0 ou 1, que resultariam em erros.
    return (prediction - label) / (prediction * (1 - prediction))


def categoricalCrossEntropy(predictions, labels):
    # usar para multiclassificação
    return -np.sum(labels * np.log(predictions)) / predictions.shape[0] # dividir por predictions.shape[0] tem o intuito de normalizar o valor da perda pelo número de amostras, tornando a perda independente do tamanho do batch


def categoricalEntropyDerivative(predictions, labels):
    epsilon=1e-12
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    return predictions - labels



costFunctions = {
    "MSE": (meanSquaredError, mseDerivative),
    "BINARY_ENTROPY": (binaryCrossEntropy, binaryEntropyDerivative),
    "CATEGORICAL_ENTROPY": (categoricalCrossEntropy, categoricalEntropyDerivative)
}

#init Layers

def initLayers(layers, attrNum):
    """
    Função para inicializar as camadas da rede neural

    Recebe:

        Layers: array de tuplas. Cada tupla representa uma camada da rede neural e possui o formato (númeroDeNeurônios, idFunçãoDeAtivação)

        attrNum: número de atributos da camada de input

    Retorna:

        initialized_layers: uma lista contendo dicionários de parâmetros para cada camada
            - weights: matriz de pesos da camada
            - activation: função de ativação da camada
            - derivation: função de derivação da camada
    """

    # inicializa prevLayerNeurons com o número de neurônios da camada de input
    prevLayerNeurons = attrNum

    # inicializa lista de camadas vazia
    initialized_layers = []

    # Itera por todas as camadas
    for i, layer in enumerate(layers):

        # inicializa dicionário de parâmetros
        layerParams = {}

        # Extrai número de neurônios e função de ativação da camada
        neuronNum, activation = layer

        # Gera a matriz de pesos da camada. Um row por neurônio, contendo um peso para cada neurônio da camada anterior + 1 para o bias
        #layerParams["weights"] = np.random.randn(
        #    neuronNum, prevLayerNeurons + 1)
        layerParams["weights"] = np.random.uniform(low=-0.33, high=0.33,size=(neuronNum,prevLayerNeurons + 1))
        
        # Extrai funções de ativação e derivação da camada
        activationFunction, derivateFunction = activationFunctions[activation]
        layerParams["activation"] = activationFunction
        layerParams["derivation"] = derivateFunction

        # Atualiza prevLayerNeurons com o número de neurônios da camada atual
        prevLayerNeurons = neuronNum

        # Guarda os parâmetros da camada
        initialized_layers.append(layerParams)

    return initialized_layers

# Backpropagation

def backpropagation(layers, costD, learningRate):
    """

    Implementa a operação da retropropagação para toda a rede

    Recebe: 

        layers: lista contendo pesos e funções de ativação/derivadas. Ver initLayers para mais detalhes

        costD: derivada da função de custo, já calculada

        learningRate: parâmetro de learningRate definido inicialmente

    Retorna:

        - Lista contendo os pesos ajustados após a retropropagação
    """

    # inicializa nextLayerWeights e nextLayerErrorSignals com None (Começa pela última camada)
    nextLayerWeights = None
    nextLayerErrorSignals = None

    # Inicializa uma lista de pesos ajustados
    adjustedWeights = []

    # itera pelas camadas (começando pela última e indo até a primeira)
    for layerParams in reversed(layers):

        # extrai valores intermediários produzidos durante a forwardPass pela camada atual
        intermediateValues = layerParams["intermediate"]

        # Obtém pesos da camada
        weights = layerParams["weights"]

        # Obtém função para calcular derivada da função de ativação da camada
        activationDerivative = layerParams["derivation"]

        # Obtém pesos ajustados e sinais de erro da camada atual
        newWeights, errorsSignals = backpropagateLayer(
            (weights, nextLayerWeights),
            intermediateValues,
            learningRate,
            costD,
            activationDerivative,
            nextLayerErrorSignals
        )

        # Atualiza nextLayerWeights e nextLayerErrorSignals com os valores correspondentes da camada atual
        nextLayerWeights = weights
        nextLayerErrorSignals = errorsSignals

        # Insere os valores dos pesos ajustados na lista de pessoas ajustados, assegurando a ordem correta
        adjustedWeights.insert(0, newWeights)

    return adjustedWeights


def backpropagateLayer(weights, intermediateValues, learningRate, costD, activationDerivative, nextLayerErrorSignals=None):
    """ 

    Realiza a operação de backpropagação em uma camada

    Recebe:

        weights: tupla contendo os pesos da layer atual e os pesos da layer seguinte

        intermediateValues: tupla contendo os valores intermediários relevantes para a camada (inputs recebidos e combinações produzidas)

        learningRate: parâmetro de learningRate

        costD: valor calculado para a derivada da função de custo

        activationDerivative: função para cálculo da derivada da função de ativação

        nextLayerErrorSignals: Error signals da camada seguinte

    Retorna:

        newWeights: os pesos ajustados após o processo de retropropagação

        errorSignals: os sinais de erro produzidos na camada, que serão propagados para a camada anterior na próxima etapa
    """

    # Extrai pesos da camada atual e da camada seguinte
    currentWeights, nextLayerWeights = weights

    # extrai inputs recebidos pela camada atual e combinações lineares que produziu
    layerInput, combinations = intermediateValues

    # calcula a derivada da função de ativação da camada
    activationD = activationDerivative(combinations)

    # Se nextLayerErrorSignals é None, calcula o errorSignals para a camada de output. Do contrário, propaga o error signal para as camadas anteriores
    if nextLayerErrorSignals is None:
        # camada de output
        errorSignals = costD * activationD
    else:
        # camadas ocultas

        # propaga o ErrorSignal da camada seguinte para a camada atual, considerando os pesos das camada seguinte
        propagatedErrorSignals = np.dot(
            nextLayerErrorSignals, nextLayerWeights[:, 1:])

        # Calcula o errorSignal da camada atual, considerando a derivada da função de ativação
        errorSignals = propagatedErrorSignals * activationD

    # calcula o gradiente da camada atual e multiplica pela learningRate
    gradients = np.outer(errorSignals, layerInput) * learningRate

    # obtém os pesos da camada atual após a retropropagação
    newWeights = currentWeights - gradients

    return newWeights, errorSignals

# RNA

def prepareInput(observation):
    """
    Inclui X0 = 1 no array de observações para multiplicar pelo BIAS.

    observation consiste em um array contendo os atributos de uma observação
    """
    return np.concatenate(([1], observation))


def forwardPass(input, weights, activationFunction):
    """
    Realizar um forward pass para uma camada

    Recebe:

        input: Array contendo as entradas da camada (atributos do input ou ativações da camada anterior)

        weights: matriz contendo os pesos de cada neurônio da camada

        activationFunction: função de ativação definida para uso na camada

    Retorna:

        activation: os valores calculados para a ativação dos neurônios

        combination: os valores calculados na combinação linear dos neurônios
    """

    combination = np.dot(input, weights.T)
    activation = activationFunction(combination)

    return activation, combination


def rna(input, layers):
    """
    Implementa a etapa de forward propagation da rede neural

    Recebe:

        input: Array contendo os atributos da observação para processamento

        layers: lista contendo dicionário com parâmetros de cada camada. Ver initLayers.

    Retorna:

        activations os valores de ativação calculados pela camada de output

    """

    # inicializa prevActivations com os valores da camada de input
    prevActivations = input

    # iteração pelas camadas da rede neural
    for i in range(len(layers)):

        # extrai parâmetros da camada
        layerParams = layers[i]

        # Extrai os pesos da camada
        layerWeights = layerParams["weights"]

        # Extrai a função de ativação da camada
        activationFunction = layerParams["activation"]

        layerInput = prepareInput(prevActivations)

        # Realiza a forwardPass da camada
        activations, combinations = forwardPass(
            layerInput, layerWeights, activationFunction)

        # Guarda os valores intermediários produzidos
        layerParams["intermediate"] = (layerInput, combinations)

        # atualiza prevActivations para os valores de ativação produzidos nessa camada
        prevActivations = activations

    return activations


# Train

def train(epochs, learningRate, layers, observations, labels, costF):
    """

    Implementa o algoritmo de treinamento da rede neural.

    Recebe:

        epochs: número de épocas para treinamento

        learningRate: valor do parâmetro de learningRate

        layers: lista com os parâmetros de cada camada da rede. Ver initLayers para mais detalhes.

        observations: lista de tuplas contendo observações para treinamento. Cada tupla tem dois elementos: ([atributosDaObservação, label])
            Exemplo:
            observations = [
                ([0, 0], 0),
                ([0, 1], 1),
                ([1, 0], 1),
                ([1, 1], 1),
                ]

        costF: identificador para função de custo. Valores válidos:
            MSE
            BINARY_ENTROPY
            CATEGORICAL_ENTROPY
    """

    error = []
    
    # extrai a função de custo e sua derivada
    costFunction, costDerivative = costFunctions[costF]

    # itera pela quantidade de épocas definida
    for n in range(epochs):

        predictions = []

        # itera pelas observações
        for observation, label in zip(observations, labels):

            # formata observations e labels como arrays
            observation = np.array(observation)
            label = np.array(label)

            # Repassa os atributos e parâmetros das camadas para a rede neural e obtém uma predição
            prediction = rna(observation, layers)
            predictions.append(prediction)

            # Calcula a derivada do custo para a observação corrente
            costD = costDerivative(prediction, label)

            # Obtém os pesos ajustados através da retropropagação
            adjustedWeights = backpropagation(
                layers, costD, learningRate)

            # Atualiza os valores dos pesos usando os valores ajustados

            for i, layer in enumerate(layers):
                layer["weights"] = adjustedWeights[i]

        # invoca a função de custo
        predictions = np.array(predictions)
        cost = costFunction(predictions, labels)
        print(cost)
        error.append(cost)

    #
    plt.plot(error)
    if (costF == "MSE"):
        plt.title("regressão - treino")
    elif (costF == "BINARY_ENTROPY"):
        plt.title("classificação binária - treino")
    else:
        plt.title("classificação multiclasse - treino")
    plt.xlabel("epoca")
    plt.ylabel("custo")
    plt.show()

    return layers

Carrega dados do dataset:

In [ ]:
DATASET_PATH = '/content/penguins.csv'
data = np.genfromtxt(DATASET_PATH, delimiter=',', skip_header=1)

Separa features e labels:

In [ ]:
features = data[:, :-3]
labels = data[:, -3:]

Separa dados para treinamento e para validação:

In [ ]:
featuresTrain, featuresVal, labelsTrain, labelsVal = splitData(features, labels)

Normaliza features:

In [ ]:
normalized_features, feature_scaler = normalize(featuresTrain)

Define a estrutura da rede, funções de ativação, funções de custo, épocas e learning rate:

In [ ]:
# Obtém a quantidade de neurons da camada de input
INPUT_NEURONS = len(normalized_features[0])

# Define a estrutura da rede e funções de ativação
layers = [
    (INPUT_NEURONS, 'RELU'),
    #(INPUT_NEURONS, 'RELU'),
    (3, 'SOFTMAX')
]

# Definição função de custo, épocas e learning rate
COSTF = "CATEGORICAL_ENTROPY"
EPOCHS = 100
LEARNING_RATE = 0.001

# Inicializa os pesos e as funções das camadas:

layers = initLayers(layers, INPUT_NEURONS)

Dispara o treinamento da rede:

In [ ]:
trainedParams = train(EPOCHS, LEARNING_RATE, layers, normalized_features, labelsTrain, COSTF)

Normaliza as features para validação:

In [ ]:
test_normalized_features = feature_scaler.transform(featuresVal)


Testa a rede treinada:

In [ ]:
predictions = []
for i, observation in enumerate(test_normalized_features):

  input = observation
  prediction = rna(input, trainedParams)

  one_hot_prediction = np.zeros_like(prediction)
  one_hot_prediction[np.argmax(prediction)] = 1

  predictions.append(one_hot_prediction)

Obtém métricas de avaliação:

In [ ]:
metrics = evaluate_multiclass_with_one_hot(labelsVal, predictions)